In [1]:
# 1. Create the graph for Zachary’s karate club as follows
import dgl
import numpy as np

def build_karate_club_graph():
    # All 78 edges are stored in two numpy arrays. One for source endpoints
    # while the other for destination endpoints.
    src = np.array([1, 2, 2, 3, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, 8, 9, 10, 10,
        10, 11, 12, 12, 13, 13, 13, 13, 16, 16, 17, 17, 19, 19, 21, 21,
        25, 25, 27, 27, 27, 28, 29, 29, 30, 30, 31, 31, 31, 31, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33, 33, 33, 33, 33])
    dst = np.array([0, 0, 1, 0, 1, 2, 0, 0, 0, 4, 5, 0, 1, 2, 3, 0, 2, 2, 0, 4,
        5, 0, 0, 3, 0, 1, 2, 3, 5, 6, 0, 1, 0, 1, 0, 1, 23, 24, 2, 23,
        24, 2, 23, 26, 1, 8, 0, 24, 25, 28, 2, 8, 14, 15, 18, 20, 22, 23,
        29, 30, 31, 8, 9, 13, 14, 15, 18, 19, 20, 22, 23, 26, 27, 28, 29, 30,
        31, 32])
    # Edges are directional in DGL; Make them bi-directional.
    u = np.concatenate([src, dst])
    v = np.concatenate([dst, src])
    # Construct a DGLGraph
    return dgl.DGLGraph((u, v))

# Print out the number of nodes and edges in our newly constructed graph
G = build_karate_club_graph()
print('We have %d nodes.' % G.number_of_nodes())
print('We have %d edges.' % G.number_of_edges())

Using backend: pytorch
We have 34 nodes.
We have 156 edges.


In [6]:
# Visualize the graph by converting it to a networkx graph
import networkx as nx
# Since the actual graph is undirected, we convert it for visualization
# purpose.
nx_G = G.to_networkx().to_undirected()
# Kamada-Kawaii layout usually looks pretty for arbitrary graphs
pos = nx.kamada_kawai_layout(nx_G)
nx.draw(nx_G, pos, with_labels=True, node_color=[[.7, .7, .7]])

In [11]:
# 2. Assign features to nodes or edges
# batches node features along the first dimension. The code below adds the learnable
# embeddings for all nodes:

import torch
import torch.nn as nn
import torch.nn.functional as F

embed = nn.Embedding(34, 5)  # 34 nodes with embedding dim equal to 5
G.ndata['feat'] = embed.weight

# Print out the node features to verify
# print out node 2's input feature
print(G.ndata['feat'][2])

# print out node 10 and 11's input features
print(G.ndata['feat'][[10, 11]])

tensor([-0.5428, -0.9428,  1.4471, -0.9325, -0.5510], grad_fn=<SelectBackward>)
tensor([[ 0.2472, -0.6637,  0.3539,  0.4138, -0.2533],
        [-0.0440,  0.6293, -0.1125, -0.4062, -1.3196]],
       grad_fn=<IndexBackward>)


In [12]:
# 3. Define a Graph Convolutional Network (GCN)
from dgl.nn.pytorch import GraphConv
# Define a deeper GCN model that contains two GCN layers:
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, hidden_size)
        self.conv2 = GraphConv(hidden_size, num_classes)

    def forward(self, g, inputs):
        h = self.conv1(g, inputs)
        h = torch.relu(h)
        h = self.conv2(g, h)
        return h

# The first layer transforms input features of size of 5 to a hidden size of 5.
# The second layer transforms the hidden layer and produces output features of
# size 2, corresponding to the two groups of the karate club.
net = GCN(5, 5, 2)

In [13]:
# 4. Data preparation and initialization
inputs = embed.weight
labeled_nodes = torch.tensor([0, 33])  # only the instructor and the president nodes are labeled
labels = torch.tensor([0, 1])  # their labels are different

In [14]:
# 5. Train then visualize
import itertools

optimizer = torch.optim.Adam(itertools.chain(net.parameters(), embed.parameters()), lr=0.01)
all_logits = []
for epoch in range(50):
    logits = net(G, inputs)
    # we save the logits for visualization later
    all_logits.append(logits.detach())
    logp = F.log_softmax(logits, 1)
    # we only compute loss for labeled nodes
    loss = F.nll_loss(logp[labeled_nodes], labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print('Epoch %d | Loss: %.4f' % (epoch, loss.item()))

Epoch 0 | Loss: 0.9354
Epoch 1 | Loss: 0.8680
Epoch 2 | Loss: 0.8056
Epoch 3 | Loss: 0.7482
Epoch 4 | Loss: 0.6957
Epoch 5 | Loss: 0.6530
Epoch 6 | Loss: 0.6164
Epoch 7 | Loss: 0.5869
Epoch 8 | Loss: 0.5651
Epoch 9 | Loss: 0.5474
Epoch 10 | Loss: 0.5278
Epoch 11 | Loss: 0.5083
Epoch 12 | Loss: 0.4908
Epoch 13 | Loss: 0.4747
Epoch 14 | Loss: 0.4584
Epoch 15 | Loss: 0.4417
Epoch 16 | Loss: 0.4248
Epoch 17 | Loss: 0.4074
Epoch 18 | Loss: 0.3896
Epoch 19 | Loss: 0.3713
Epoch 20 | Loss: 0.3522
Epoch 21 | Loss: 0.3329
Epoch 22 | Loss: 0.3135
Epoch 23 | Loss: 0.2944
Epoch 24 | Loss: 0.2756
Epoch 25 | Loss: 0.2573
Epoch 26 | Loss: 0.2394
Epoch 27 | Loss: 0.2221
Epoch 28 | Loss: 0.2054
Epoch 29 | Loss: 0.1893
Epoch 30 | Loss: 0.1739
Epoch 31 | Loss: 0.1592
Epoch 32 | Loss: 0.1453
Epoch 33 | Loss: 0.1325
Epoch 34 | Loss: 0.1209
Epoch 35 | Loss: 0.1100
Epoch 36 | Loss: 0.0999
Epoch 37 | Loss: 0.0905
Epoch 38 | Loss: 0.0818
Epoch 39 | Loss: 0.0738
Epoch 40 | Loss: 0.0666
Epoch 41 | Loss: 0.0600
Ep

In [31]:
# plotting the output feature in a 2D space
import matplotlib.animation as animation
import matplotlib.pyplot as plt

def draw(i):
    cls1color = '#00FFFF'
    cls2color = '#FF00FF'
    pos = {}
    colors = []
    for v in range(34):
        pos[v] = all_logits[i][v].numpy()
        cls = pos[v].argmax()
        colors.append(cls1color if cls else cls2color)
    ax.cla()
    ax.axis('off')
    ax.set_title('Epoch: %d' % i)
    nx.draw_networkx(nx_G.to_undirected(), pos, node_color=colors,
            with_labels=True, node_size=300, ax=ax)

fig = plt.figure(dpi=150)
fig.clf()
ax = fig.subplots()
draw(0)  # draw the prediction of the first epoch
# plt.close()

In [32]:
ani = animation.FuncAnimation(fig, draw, frames=len(all_logits), interval=200)